In [2]:
import pandas as pd
import os
import copy
import unittest
import string
import re

def read_file(trainFile):
    pwd = os.getcwd()
    os.chdir(os.path.dirname(trainFile))
    file1 = open(trainFile, 'r') 
    lines = file1.read().splitlines()
    return lines

In [3]:
lines = read_file("C:/Code/advent/2021_day16_input.txt")

In [4]:
lines_example1 = read_file("C:/Code/advent/2021_day16_example1.txt")
lines_example1

['D2FE28',
 '38006F45291200',
 '8A004A801A8002F478',
 '620080001611562C8802118E34',
 'C0015000016115A2E0802F182340',
 'A0016C880162017C3686B18A3D4780']

In [5]:
lines_example2 = read_file("C:/Code/advent/2021_day16_example2.txt")
lines_example2

['C200B40A82',
 '04005AC33890',
 '880086C3E88112',
 'CE00C43D881120',
 'D8005AC2A8F0',
 'F600BC2D8F',
 '9C005AC2F8F0',
 '9C0141080250320F1802104A08']

In [6]:
value_map = {
'0' : '0000',
'1' : '0001',
'2' : '0010',
'3' : '0011',
'4' : '0100',
'5' : '0101',
'6' : '0110',
'7' : '0111',
'8' : '1000',
'9' : '1001',
'A' : '1010',
'B' : '1011',
'C' : '1100',
'D' : '1101',
'E' : '1110',
'F' : '1111'
}

def to_bits(input):
    result = []
    for c in input:
        bits = map(int, value_map[c])
        for bit in bits:
            result.append(bit)
    return result

In [24]:
def to_int(input):
    value = 0
    for bit in input:
        value *= 2
        value += bit
    return value

def read_integer(input, position):
    value = 0
    while position < len(input):
        value *= 16
        done = input[position] == 0
        value += to_int(input[position+1:position+5]) 
        position += 5
        if done:
            return (position, value)

def read_packet_list(input, debug):
    position = 0
    results = []
    while position < len(input):
        result = read_packet(input[position:], debug)
        position += result[0]
        results.append(result)
    return results

def calculate_value(subresults, type_id):
    numbers = [s[2] for s in subresults]
    result = None
    if type_id == 0:
        result = sum(numbers)
        print('sum', numbers, result)        
    elif type_id == 1:
        result = 1
        for number in numbers:
            result *= number
        print('product', numbers, result) 
    elif type_id == 2:
        result = min(numbers)
        print('min', numbers, result)        
    elif type_id == 3:
        result = max(numbers)
        print('max', numbers, result)        
    elif type_id == 5:        
        result = 1 if numbers[0] > numbers[1] else 0
        print('greater', numbers)
    elif type_id == 6:        
        result = 1 if numbers[0] < numbers[1] else 0
        print('less', numbers, result)        
    elif type_id == 7:        
        result = 1 if numbers[0] == numbers[1] else 0
        print('equal', numbers, result)
    return result

def read_packet(input, debug):
    position = 0
    if debug:
        print('read_packet', input)
    result = []
    value = None
    version = to_int(input[position:position+3])
    result.append(version)
    position += 3
    type_id = to_int(input[position:position+3])
    position += 3
    if type_id == 4:
        literal = read_integer(input, position)
        if debug:
            print('literal',version, 'value', literal)
        position = literal[0]
        value = literal[1]
    else:
        length_type_id = input[position]
        position += 1
        if length_type_id == 0:
            total_length = to_int(input[position:position+15])
            position += 15
            if debug:
                print('total length operator',version, total_length)
            subpacket = input[position:position+total_length]
            if debug:
                print(subpacket)
            position += total_length
            subresults = read_packet_list(subpacket, debug)
            for subresult in subresults:
                for version in subresult[1]:
                    result.append(version)
            value = calculate_value(subresults, type_id)
        else:
            total_packets = to_int(input[position:position+11])
            position += 11
            if debug:
                print('total packet operator',version, total_packets)
            subresults = []
            for i in range(total_packets):
                remainder = input[position:]
                subresult = read_packet(remainder, debug)
                subresults.append(subresult)
                position += subresult[0]
                for version in subresult[1]:
                    result.append(version)
            value = calculate_value(subresults, type_id)
    
    if debug:
        print(result, value)
    return (position, result, value)

def decode(input, debug):
    print(input)
    bits = to_bits(input)
    result = read_packet(bits, debug)
    print(result)
    return result

def calculate_part_one(input, debug):
    for line in input:
        result = decode(line, debug)
        print('part 1',sum(result[1]))
        print('part 2',result[2])
        print("-------------------")
    
calculate_part_one(lines_example1, True)

D2FE28
read_packet [1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0]
literal 6 value (21, 2021)
[6] 2021
(21, [6], 2021)
part 1 6
part 2 2021
-------------------
38006F45291200
read_packet [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
total length operator 1 27
[1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0]
read_packet [1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0]
literal 6 value (11, 10)
[6] 10
read_packet [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0]
literal 2 value (16, 20)
[2] 20
less [10, 20] 1
[1, 6, 2] 1
(49, [1, 6, 2], 1)
part 1 9
part 2 1
-------------------
8A004A801A8002F478
read_packet [1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 

In [25]:
# 653 too low :(

# 101725040702 too low
# 110434737925
calculate_part_one(lines, False)

020D78804D397973DB5B934D9280CC9F43080286957D9F60923592619D3230047C0109763976295356007365B37539ADE687F333EA8469200B666F5DC84E80232FC2C91B8490041332EB4006C4759775933530052C0119FAA7CB6ED57B9BBFBDC153004B0024299B490E537AFE3DA069EC507800370980F96F924A4F1E0495F691259198031C95AEF587B85B254F49C27AA2640082490F4B0F9802B2CFDA0094D5FB5D626E32B16D300565398DC6AFF600A080371BA12C1900042A37C398490F67BDDB131802928F5A009080351DA1FC441006A3C46C82020084FC1BE07CEA298029A008CCF08E5ED4689FD73BAA4510C009981C20056E2E4FAACA36000A10600D45A8750CC8010989716A299002171E634439200B47001009C749C7591BD7D0431002A4A73029866200F1277D7D8570043123A976AD72FFBD9CC80501A00AE677F5A43D8DB54D5FDECB7C8DEB0C77F8683005FC0109FCE7C89252E72693370545007A29C5B832E017CFF3E6B262126E7298FA1CC4A072E0054F5FBECC06671FE7D2C802359B56A0040245924585400F40313580B9B10031C00A500354009100300081D50028C00C1002C005BA300204008200FB50033F70028001FE60053A7E93957E1D09940209B7195A56BCC75AE7F18D46E273882402CCD006A600084C1D8ED0E8401D8A90BE12CCF2F4C4ADA602013BC401

In [19]:
calculate_part_one(lines_example2, False)

C200B40A82
sum [1, 2]
(40, [6, 6, 2], 3)
part 1 14
part 2 3
-------------------
04005AC33890
product [6, 9]
(44, [0, 5, 3], 54)
part 1 8
part 2 54
-------------------
880086C3E88112
min [7, 8, 9]
(55, [4, 5, 6, 0], 7)
part 1 15
part 2 7
-------------------
CE00C43D881120
max [7, 8, 9]
(51, [6, 0, 5, 0], 9)
part 1 11
part 2 9
-------------------
D8005AC2A8F0
less [5, 15]
(44, [6, 5, 2], 1)
part 1 13
part 2 1
-------------------
F600BC2D8F
greater [5, 15]
(40, [7, 7, 5], 0)
part 1 19
part 2 0
-------------------
9C005AC2F8F0
equal [5, 15]
(44, [4, 5, 7], 0)
part 1 16
part 2 0
-------------------
9C0141080250320F1802104A08
sum [1, 3]
product [2, 2]
equal [4, 4]
(102, [4, 2, 2, 4, 6, 0, 2], 1)
part 1 20
part 2 1
-------------------
